**About** : This notebook is used to validate models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
torch.cuda.get_device_name(0)

In [ ]:
import os
import re
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
pd.set_option('display.max_columns', 500)

In [ ]:
from params import *

from data.preparation import *
from data.dataset import SignDataset

from utils.metrics import *
from utils.plots import *
from utils.logger import Config

from inference.main import kfold_inference_val

### Blend

In [ ]:
df = prepare_data(DATA_PATH, "")

In [ ]:
# EXP_FOLDER = "../logs/2023-04-17/42/"  # 0.7265 / x5 0.7273 / MTx10 0.7274
# EXP_FOLDER = "../logs/2023-04-18/10/"  # 0.7245 / x5 0.7247 / MTx10 0.7249
# EXP_FOLDER = "../logs/2023-04-19/6/"  # 0.7245 / x10 0.7254 / MTx10 0.7254
# EXP_FOLDER = "../logs/2023-04-20/18/"  # 0.7247 / x10 0.7254 / MTx10 0.7256
# EXP_FOLDER = "../logs/2023-04-20/19/"  # 0.7250 / x10 0.7257 / MTx10 0.7254
# EXP_FOLDER = "../logs/2023-04-20/21/"  # 0.7243 / x10 0.7246 / MTx10 0.7244

FILES = [  #  CV 0.7279 - LB 0.78
    "../logs/2023-04-14/37/pred_oof_dist.npy",  # 0.7225 / Dist 0.7196 (-0.003)
    "../logs/2023-04-15/7/pred_oof_dist.npy",  # 0.7228 / Dist 0.7176 (-0.005)
]

FILES = [  # CV 0.7340 - LB 0.78
    "../logs/2023-04-20/19/pred_oof_inf.npy",  # 0.7257
    "../logs/2023-04-20/18/pred_oof_mt.npy",  # 0.7254
]


# FILES = [  # CV 0.7351
# #     "../logs/2023-04-22/6/pred_oof_mt.npy",  # 0.7298
#     "../logs/2023-04-22/7/pred_oof_mt.npy",  # 0.7310
# #     "../logs/2023-04-22/2/pred_oof.npy",  # 0.7310
# ]

# WEIGHTS = [1, 1]

try:
    pred_oof = np.average([np.load(f) for f in FILES], weights=WEIGHTS, axis=0)
    print('- Weighted average')
except:
    print('- Simple average')
    pred_oof = np.mean([np.load(f) for f in FILES], 0)
df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"\n-> CV acc : {score:.4f}")

## Expes

In [ ]:
EXP_FOLDER = "../logs/2023-04-21/31/"  # 0.7267
# EXP_FOLDER = "../logs/2023-04-22/2/"  # 0.7310

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [ ]:
df = prepare_data(DATA_PATH, config.processed_folder)

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [ ]:
pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")
df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

In [ ]:
# df = df[df['participant_id'] != 29302].reset_index(drop=True)
# score = accuracy(df['target'], df['pred'])
# print(f"-> CV acc : {score:.4f}")

In [ ]:
df['error'] = (df['target'] != df['pred'])

In [ ]:
dfg = df.groupby('participant_id').agg(['mean', 'count', 'sum'])[['error']]

In [ ]:
dfg.sort_values(('error',  'mean'))

In [ ]:
dfg = df.groupby('sign').agg('mean')[['error']].sort_values('error', ascending=False).T
dfg

In [ ]:
classes = json.load(open(DATA_PATH + "sign_to_prediction_index_map.json", "r"))
classes = list(classes.keys())

In [ ]:
cm = confusion_matrix(df['target'], df['pred'], normalize=None)

In [ ]:
for i in tqdm(range(len(classes))):  # gt
    for j in range(len(classes)):
        n = cm[i, j]
        if n > 50 and i != j:
            s = f"{classes[i]} predicted as {classes[j]} :".ljust(32)
            print(f"{s} {n} / {cm[i].sum()}")

In [ ]:
# plt.figure(figsize=(50, 50))
# plot_confusion_matrix(df['pred'], df['target'], display_labels=None)

### Inf val

In [ ]:
EXP_FOLDER = "../logs/2023-04-17/42/"  # 0.7265 / x5 0.7273 / MTx10 0.7274
EXP_FOLDER = "../logs/2023-04-18/10/"  # 0.7245 / x5 0.7247 / MTx10 0.7249
# EXP_FOLDER = "../logs/2023-04-19/6/"  # 0.7245 / x10 0.7254 / MTx10 0.7254
# EXP_FOLDER = "../logs/2023-04-20/18/"  # 0.7247 / x10 0.7254 / MTx10 0.7256
# EXP_FOLDER = "../logs/2023-04-20/19/"  # 0.7250 / x10 0.7257 / MTx10 0.7254
# EXP_FOLDER = "../logs/2023-04-20/21/"  # 0.7243 / x10 0.7246 / MTx10 0.7244

EXP_FOLDER = "../logs/2023-04-21/9/"  # 0.7262 / x10 0.7269 / MTx10 0.7268

EXP_FOLDER = "../logs/2023-04-21/31/"  # 0.7267 / x10 0.7278 / MTx10 0.7278

EXP_FOLDER = "../logs/2023-04-22/6/"   # 0.7294 / x10 0.7296 / MTx10 0.7298
# EXP_FOLDER = "../logs/2023-04-22/7/"   # 0.7295 / x10 0.7303 / MTx10 0.7310

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))
df = prepare_data(DATA_PATH, config.processed_folder)

if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])
    
try:
    pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")
    df['pred'] = pred_oof.argmax(-1)

    score = accuracy(df['target'], pred_oof)
    print(f"-> CV acc : {score:.4f}")
except:
    df = df[df['fold'] == 0].reset_index(drop=True)
    
    pred_oof = np.load(EXP_FOLDER + "pred_val_0.npy")
    df['pred'] = pred_oof.argmax(-1)

    score = accuracy(df['target'], pred_oof)
    print(f"-> Fold 0 acc : {score:.4f}")

In [ ]:
pred_oof = kfold_inference_val(
    df,
    EXP_FOLDER,
    debug=False,
    save=True,
    use_fp16=True,
    use_mt=False,
    distilled=False,
    n_soup=10,
)

In [ ]:
# pred = (pred_oof + np.load(EXP_FOLDER + "pred_val_0.npy")) / 2
# accuracy(df['target'], pred)

# df_val['pred_flip'] = pred.argmax(-1)
# df_val['error_flip'] = (df_val['pred_flip'] != df_val['target'])

# dfg = df_val.groupby('sign')[["error", "error_flip"]].mean()
# dfg["delta"] = dfg["error"] - dfg["error_flip"]

# dfg.sort_values('delta').T

# [classes.index(c) for c in dfg[dfg['delta'] < 0.01].index]

# [classes.index(c) for c in dfg[dfg['delta'] < -0.05].index]

### Inf train

In [ ]:
EXP_FOLDER = "../logs/2023-04-11/27/"

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))
df = prepare_data(DATA_PATH, config.processed_folder)

if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [ ]:
pred_oof = kfold_inference_val(
    df,
    EXP_FOLDER,
    debug=False,
    save=False,
    use_fp16=True,
    train=True
)

In [ ]:
np.save(EXP_FOLDER + "pred_oof_train.npy", pred_oof)

#### Checks

In [ ]:
df['pred_0'] = pred_oof[0].argmax(-1)
df_val = df[df['fold'] != 0].reset_index(drop=True)

In [ ]:
df_err = df_val[(df_val['target'] != df_val['pred_0'])].reset_index(drop=True)

In [ ]:
dataset = SignDataset(df_err, max_len=None, train=False)

In [ ]:
for i in tqdm(range(len(dataset))):
#     i = 92284
    data = dataset[i]
    
#     for k in data.keys():
#         print(k, data[k].size())
    
    print(df_err['sequence_id'][i], "- pred :", classes[df_err['pred_0'][i]], " - truth :", df_err['sign'][i])
    plot_sample_with_edges(data, n_frames=4, figsize=(10, 10), show_text=False)

    if i > 10:
        break

Done ! 